# 01. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Importing Dataframes

In [2]:
# Define path
path = r'C:\Users\Lex\OneDrive\Data Analytics\Data Immersion - Python\03-2022 Instacart Basket Analysis'

In [3]:
# Import customers dataframe
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [ ]:
# Import most recent orders_products dataframe
df_ordsprods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_4.8B.pkl'))

# 03. Data Wrangling

In [5]:
df_cust.shape

(206209, 10)

Adjusting columns

In [12]:
# Create Name object
Name = df_cust['First Name'] + (' ') + df_cust['Surnam']

In [22]:
# Remove Surname column
df_cust = df_cust.drop(columns = ['Surnam'])

In [ ]:
# Change First Name column to include full name
df_cust['First Name'] = Name

In [24]:
# Rename First Name column to Name
df_cust = df_cust.rename(columns = {'First Name' : 'Name'})

In [15]:
# Rename STATE to State
df_cust.rename(columns = {'STATE' : 'State'}, inplace = True)

In [25]:
df_cust.head()

,user_id,Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Performing data quality checks

In [26]:
# Mixed-Type data check
for col in df_cust.columns.tolist():
    weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_cust[weird]) > 0:
        print (col)

Name


In [28]:
df_cust['Name'].dtype

dtype('O')

In [29]:
# Change Name column to string
df_cust['Name'] = df_cust['Name'].astype('str')

In [30]:
df_cust['Name'].dtype

dtype('O')

In [31]:
# Checking again
for col in df_cust.columns.tolist():
    weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_cust[weird]) > 0:
        print (col)

In [32]:
# Missing Values check
df_cust.isnull().sum()

user_id         0
Name            0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [33]:
# Duplicates check
df_duplicates = df_cust[df_cust.duplicated()]

In [34]:
df_duplicates

,user_id,Name,Gender,State,Age,date_joined,n_dependants,fam_status,income


Impromptu wrangling of df_ordsprods

In [37]:
pd.options.display.max_columns = None

In [38]:
df_ordsprods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spend,spending_flag,avg_days_since_prior_order,frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77.0,7.0,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23.0,19.0,4.4,both,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7,both,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both,Low-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer


In [40]:
# Dropping _merge and busiest_day. Do I really need to keep max_order, avg_spend or avg_days_since_prior_order?
df_ordsprods = df_ordsprods.drop(columns = ['_merge', 'busiest_day'])

In [41]:
# Risking it.
df_ordsprods = df_ordsprods.drop(columns = ['max_order', 'avg_spend', 'avg_days_since_prior_order'])

# 04. Merging Dataframes

In [43]:
df_cust['user_id'].dtype

dtype('int64')

In [44]:
df_ordsprods['user_id'].dtype

dtype('int64')

In [45]:
# Merge test
pd.merge(df_ordsprods, df_cust, on = ['user_id'], how = 'left')

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,loyalty_flag,spending_flag,frequency_flag,Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77.0,7.0,9.0,Mid-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,Mid-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23.0,19.0,4.4,Low-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7,Low-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,Low-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32429927,2977660,206209,13,1,12,7.0,False,14197,5,1,Tomato Paste,9.0,9.0,5.6,Mid-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969
32429928,2977660,206209,13,1,12,7.0,False,38730,6,0,Brownie Crunch High Protein Bar,3.0,19.0,5.9,Mid-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969
32429929,2977660,206209,13,1,12,7.0,False,31477,7,0,High Protein Bar Chunky Peanut Butter,3.0,19.0,4.2,Low-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969
32429930,2977660,206209,13,1,12,7.0,False,6567,8,0,Chocolate Peanut Butter Protein Bar,3.0,19.0,4.9,Low-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969


In [48]:
# Merging df_ordsprods and df_cust
df_merge = df_ordsprods.merge(df_cust, on = ['user_id'], how = 'left')

In [49]:
df_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,loyalty_flag,spending_flag,frequency_flag,Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77.0,7.0,9.0,Mid-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,Mid-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23.0,19.0,4.4,Low-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23.0,19.0,4.7,Low-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,Low-range product,Regularly busy,Average orders,New customer,Low spender,Non-frequent customer,Linda Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [50]:
df_merge.tail()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,loyalty_flag,spending_flag,frequency_flag,Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
32429927,2977660,206209,13,1,12,7.0,False,14197,5,1,Tomato Paste,9.0,9.0,5.6,Mid-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969
32429928,2977660,206209,13,1,12,7.0,False,38730,6,0,Brownie Crunch High Protein Bar,3.0,19.0,5.9,Mid-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969
32429929,2977660,206209,13,1,12,7.0,False,31477,7,0,High Protein Bar Chunky Peanut Butter,3.0,19.0,4.2,Low-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969
32429930,2977660,206209,13,1,12,7.0,False,6567,8,0,Chocolate Peanut Butter Protein Bar,3.0,19.0,4.9,Low-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969
32429931,2977660,206209,13,1,12,7.0,False,22920,9,0,Roasted & Salted Shelled Pistachios,117.0,19.0,1.7,Low-range product,Busiest days,Most orders,Regular customer,Low spender,Non-frequent customer,Diana Donovan,Female,Iowa,74,9/14/2019,3,married,137969


In [52]:
# Exporting df_merge
df_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'final_dataframe_v1.pkl'))